In [21]:
#Importing the required libraries and opening the JSON file
import json
import pandas as pd
try:
    with open('C:/Users/dvive/Downloads/Project/Data/config.json', "r") as json_file:
        config_file = json_file.read()
except FileNotFoundError as error:
    print("File not found:", error)
except Exception as error:
    print("Error opening file:", error)

#Converting the JSON file to a dictionary
try:
    config_file = json.loads(config_file) 
except json.JSONDecodeError as error:
    print("Error decoding JSON:", error)
except Exception as error:
    print("Error loading config file:", error)
    
#Saving the Primary Key of the JSON file
primary_key = "sql_steps"

#Printing the details of the JSON file
print("Type of Config File : ",type(config_file))
print("Keys in Config File : ",config_file.keys())
print("The Values of sql_steps is a list: ",isinstance(config_file["sql_steps"],list))
print("Value of Key 'sql_steps' : ",config_file['sql_steps'])

#Function to check if the key exists in the JSON file
config_file.setdefault(primary_key,"Key Not found") 
#Function to check if the datatype of the variable matches the desired datatype 
def validate_datatype(var,datatype):
    if type(var) == datatype:
        return f"Datatype {datatype} Matched"
    else:
        return f"Datatype {datatype} not Matched"
    
#Calling the function to check the datatype of the variable    
print(validate_datatype(config_file,dict))

#Function to Validate Data in the config JSON file
def check_keys(*args):
    for i in range(len(config_file[primary_key])):
        try:
            #Saving repeating logic to variables
            sql_num_value = config_file[primary_key][i].get(args[0])
            action_value = config_file[primary_key][i].get(args[1])
            action_parsed_value = config_file[primary_key][i].get(args[1]).split("|")[0]
            action_format_value = config_file[primary_key][i].get(args[1]).split("|")[1]
            next_sql_value = config_file[primary_key][i].get(args[2])
            print()
            #If condition to check if the keys exist in the config JSON file and converting sql_num to int
            if sql_num_value is not None and action_value is not None and next_sql_value is not None:
                print("All Keys Found in Block ",i)
                if isinstance(sql_num_value,int)==False:
                    print("sql_num exists: ",sql_num_value," Type: ",type(sql_num_value))
                    #Typecast sql_num_value to int and save changes to config_file
                    config_file[primary_key][i][args[0]] = int(sql_num_value)
                    sql_num_value = int(sql_num_value)
                    print("Type of sql_num typecasted to: ",type(sql_num_value))
            else:
                print("All Keys not Found in Block ",i)
                if  sql_num_value is None:
                    print("sql_num not Found in Block ",i)
                if  action_value is None:
                    print("action not Found in Block ",i)
                if  next_sql_value is None:
                    print("next_sql not Found in Block ",i)
            #If condition to check if the action is read or write
            if action_parsed_value not in ("read", "write"):
                print("Error : Invalid action-",action_parsed_value,"exists in Block ",i,". ")
            else:
                print("Valid action",action_parsed_value,"exists in Block ",i,". ")   
            #If condition to check if the action is read and the format is database or parquet
            if action_parsed_value=="read" and action_format_value not in ("database","parquet"):
                        print("Error : Invalid Format-",action_format_value,"exists in Block ",i,". ")
            if action_parsed_value=="read" and action_format_value in ("database","parquet"):
                print("Valid Format",action_format_value,"exists in Block ",i,". ")
            #If condition to check if the action is write and the format is parquet
            if action_parsed_value=="write" and action_format_value not in ("parquet"):
                print("Error : Invalid Format-",action_format_value,"exists in Block ",i,". ")   
            if action_parsed_value=="write" and action_format_value in ("parquet"):
                print("Valid Format",action_format_value,"exists in Block ",i,". ")
        except Exception as error:
            print(f"{error}")



#Calling the function to check the keys in the config JSON file    
check_keys("sql_num","action","next_sql")   

Type of Config File :  <class 'dict'>
Keys in Config File :  dict_keys(['sql_steps'])
The Values of sql_steps is a list:  True
Value of Key 'sql_steps' :  [{'sql_num': '1', 'action': 'read|publish', 'sql': "select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd = 'RORGCL' union select original_claim_num_cd, parent_claim_num_cd, child_claim_num_cd, claim_ordinal_value_num from ods_vw_service.mv_consume_claim_ln_reprocess_assn where claim_ln_reprocess_tp_cd != 'RORGCL' and parent_claim_num_cd != child_claim_num_cd", 'temp_view': 'consume_claim_ln_reprocess_assn', 'next_sql': '2'}, {'sql_num': '2', 'action': 'write|parquet', 'sql': 'select * from consume_claim_ln_reprocess_assn', 'location': 'sandbox/claims_reprocess/06-07/claim_ln_repr_ordinal_2/', 'mode': 'overwrite', 'next_sql': '3'}, {'sql_num': '3', 'action': 'read|parquet', 'location': 'sandbox/claims_reproce

In [27]:

import csv

def read_csv_file(location):
    data = []
    with open(location, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        for row in reader:
            data.append(dict(zip(header, row)))
    return data

def write_csv_file(location, data):
    with open(location, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def read_parquet_file(location):
    data = []
    with open(location, 'r') as f:
        reader = parquet.reader(f)
    return data

def write_parquet_file(location):
     with open(location, 'w') as f:
        writer = parquet.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def read_database(location):
    data = []
    with open(location, 'r') as f:
        reader = sql.reader(f)
        header = next(reader)
        for row in reader:
            data.append(dict(zip(header, row)))
    return data

def write_database(location, data):
    with open(location, 'w') as f:
        writer = sql.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)


def read_file(action, format):
    if action=='read':
        if format=='csv':
            read_csv_file(i["location"])
        if format=='parquet': 
            read_parquet_file(i["location"])
        if format=='database':
            read_database(i["location"])

    if action=='write':
        if format=='csv':
            write_csv_file(i["location"])
        if format=='parquet': 
            write_parquet_file(i["location"])
        if format=='database':
            write_database(i["location"])

In [28]:
for i in config_file["sql_steps"]:
    read_file(i["action"].split("|")[0],i["action"].split("|")[1])

FileNotFoundError: [Errno 2] No such file or directory: 'sandbox/claims_reprocess/06-07/claim_ln_repr_ordinal_2/'